In [1]:
#default_exp segmentation.models

In [2]:
%autosave 60 
import fastai; print(fastai.__version__)

Autosaving every 60 seconds
1.0.59.dev0


In [3]:
#export
from fastai.vision import *

In [4]:
torch.cuda.set_device(1)

### data

In [63]:
from local.segmentation.dataset import SemanticSegmentationData
from local.segmentation.metrics import *
from local.segmentation.losses_multilabel import *
# test data creation
def _get_test_data(size=320):
    PATH = Path("/home/turgutluk/.fastai/data/camvid")
    IMAGES = "images"
    MASKS = "labels"
    CODES = "codes.txt"
    TRAIN, VALID, TEST = "train.txt", "valid.txt", "test.txt"
    ssdata = SemanticSegmentationData(PATH, IMAGES, MASKS, CODES, TRAIN,
                                      VALID, TEST, sample_size=None, bs=4, size=size)
    return ssdata.get_data()

In [64]:
data = _get_test_data()

### configs

In [6]:
#export
default_configs = {}
model_funcs = {}
splits = {}

In [7]:
#export
def get_model(name, data, config):
    "Get model given name, data and config. Undefined config is defaulted."
    conf, copy_conf = default_configs[name].copy(), default_configs[name].copy()
    conf.update(config)    
    f = model_funcs[name]
    model = f(data, conf)
    split_fn = splits.get(name)
    return model, split_fn, copy_conf

In [8]:
def _test_model(model, data, split_fn):
    "camvid training test"
    learn = Learner(data, model)
    learn = learn.split(split_fn)
    learn.metrics = [partial(foreground_acc, void_code=31)]
    if config.get("pretrained"): learn.freeze()
    else: apply_init(learn.model, nn.init.kaiming_normal_)
    epochs, max_lr = 2, 3e-3
    learn.freeze_to(-1)
    learn.fit_one_cycle(epochs, max_lr)

    # stage-2
    lrs = slice(max_lr/100,max_lr/4)
    learn.freeze_to(-2)
    learn.fit_one_cycle(epochs, lrs, pct_start=0.8)

    # stage-3
    lrs = slice(max_lr/100,max_lr/4)
    learn.unfreeze()
    learn.fit_one_cycle(epochs, lrs, pct_start=0.8)
    
    res = learn.recorder.metrics[-1][0].item(); print(res)
    assert res > 0.80

### `segmentation_models_pytorch`

- Models from great repo: https://github.com/qubvel/segmentation_models.pytorch
- We will only use models and combine with fastai Learner - best of both worlds
- Pretrained on `imagenet+5k`, `imagenet`, `instagram`
- `Unet`, `FPN`, `LinkNet`, `PSPNet`


**⚠️Warning:** Models from `segmentation_models_pytorch` here don't support all image sizes! DynamicUnet does.

In [9]:
#export
import local.segmentation.segmentation_models_pytorch as smp

In [10]:
#export
def _smp_split(m:nn.Module): return (m.decoder, m.decoder)

### `smpUnet`

In [11]:
#export
smpunet_config = {"encoder_name":'resnext101_32x8d',
                "encoder_weights":'instagram',
                "decoder_use_batchnorm":True,
                "decoder_channels":(256, 128, 64, 32, 16),
                "center":False,
                "attention_type":None,
                "activation":'sigmoid',
                  
#                 classes=data.c,
                "pretrained":True
                 }

In [12]:
#export
def smpunet(data, config):
    config.pop("pretrained")
    return smp.Unet(classes=data.c, **config)

model_funcs['smpunet'] = smpunet
default_configs['smpunet'] = smpunet_config
splits['smpunet'] = _smp_split

### `smpFPN`

In [13]:
#export
smpfpn_config = {"encoder_name":'resnet34',
                "encoder_weights":'imagenet',
                "decoder_pyramid_channels":256,
                "decoder_segmentation_channels":128,
                "dropout":0.2,
                "activation":'sigmoid',
                "final_upsampling":4,
                "decoder_merge_policy":'add',
                  
#                 classes=data.c,
                "pretrained":True
                 }

In [15]:
#export
def smpfpn(data, config):
    config.pop("pretrained")
    return smp.FPN(classes=data.c, **config)

model_funcs['smpfpn'] = smpfpn
default_configs['smpfpn'] = smpfpn_config
splits['smpfpn'] = _smp_split

### `spmLinknet`

In [16]:
#export
smplinknet_config = {"encoder_name":'resnet34',
                    "encoder_weights":'imagenet',
                    "decoder_use_batchnorm":True,
                    "activation":'sigmoid',
                  
    #                 classes=data.c,
                    "pretrained":True
                     }

In [17]:
#export
def smplinknet(data, config):
    config.pop("pretrained")
    return smp.Linknet(classes=data.c, **config)

model_funcs['smplinknet'] = smplinknet
default_configs['smplinknet'] = smplinknet_config
splits['smplinknet'] = _smp_split

### `spmPSPNet`

In [18]:
#export
smppspnet_config = {"encoder_name":'resnet34',
                    "encoder_weights":'imagenet',
                    "psp_in_factor":8,
                    "psp_out_channels":512,
                    "psp_use_batchnorm":True,
                    "psp_aux_output":False,
                    "dropout":0.2,
                    "activation":'softmax',
                    
#                 classes=data.c,
                    "pretrained":True
                         }

In [19]:
#export
def smppspnet(data, config):
    config.pop("pretrained")
    return smp.PSPNet(classes=data.c, **config)

model_funcs['smppspnet'] = smppspnet
default_configs['smppspnet'] = smppspnet_config
splits['smppspnet'] = _smp_split

### `ResDUnet [18,34,50,101,152]`

In [20]:
#export
from fastai.vision.models.unet import DynamicUnet
from fastai.vision.learner import cnn_config

_body_config = {"pretrained":True} 
_unet_config = {"blur":False, "blur_final":True, "self_attention":False,
         "y_range":None, "norm_type":NormType, "last_cross":True, "bottle":False}
dunet_config = {**_body_config, **_unet_config}

In [21]:
#export
from fastai.vision.models import resnet18, resnet34, resnet50, resnet101, resnet152
from fastai.vision.models.cadene_models import model_meta

In [23]:
#export
_res_meta = model_meta[resnet18]
_res_cut, _res_split = _res_meta['cut'], _res_meta['split']

In [24]:
#export
def _resdunet(arch, data, config):
    "Returns a resdunet model for an arch from data and final config"
    pretrained = config.pop("pretrained")
    try:    size = data.train_ds[0][0].size
    except: size = next(iter(data.train_dl))[0].shape[-2:]
    body = create_body(arch, pretrained, _res_cut)
    model = DynamicUnet(body, n_classes=data.c, img_size=size, **config)
    return model

In [25]:
#export
def resdunet18(data, config): return _resdunet(resnet18, data, config)
model_funcs['resdunet18'] = resdunet18
default_configs['resdunet18'] = dunet_config
splits['resdunet18'] = _res_split

def resdunet34(data, config): return _resdunet(resnet34, data, config)
model_funcs['resdunet34'] = resdunet34
default_configs['resdunet34'] = dunet_config
splits['resdunet34'] = _res_split

def resdunet50(data, config): return _resdunet(resnet50, data, config)
model_funcs['resdunet50'] = resdunet50
default_configs['resdunet50'] = dunet_config
splits['resdunet50'] = _res_split

def resdunet101(data, config): return _resdunet(resnet101, data, config)
model_funcs['resdunet101'] = resdunet101
default_configs['resdunet101'] = dunet_config
splits['resdunet101'] = _res_split

def resdunet152(data, config): return _resdunet(resnet152, data, config)
model_funcs['resdunet152'] = resdunet152
default_configs['resdunet152'] = dunet_config
splits['resdunet152'] = _res_split

### `EfficientDUnet (TODO - Experimental)`

https://github.com/narumiruna/efficientnet-pytorch

In [20]:
from fastai.vision.models.efficientnet import *

In [21]:
def EfficientNetB1(pretrained, data): return EfficientNet.from_pretrained('efficientnet-b1', num_classes=data.c)
def EfficientNetB2(pretrained, data): return EfficientNet.from_pretrained('efficientnet-b2', num_classes=data.c)
def EfficientNetB3(pretrained, data): return EfficientNet.from_pretrained('efficientnet-b3', num_classes=data.c)
def EfficientNetB4(pretrained, data): return EfficientNet.from_pretrained('efficientnet-b4', num_classes=data.c)
def EfficientNetB5(pretrained, data): return EfficientNet.from_pretrained('efficientnet-b5', num_classes=data.c)
def EfficientNetB6(pretrained, data): return EfficientNet.from_pretrained('efficientnet-b6', num_classes=data.c)
def EfficientNetB7(pretrained, data): return EfficientNet.from_pretrained('efficientnet-b7', num_classes=data.c)

### `DenseDUnet [121,161,169, 201] (FIX - Buggy :( )`

In [34]:
# from fastai.callbacks.hooks import model_sizes
# from fastai.vision.models import densenet121, densenet161, densenet169, densenet201

In [35]:
# def _densedunet_split(m:nn.Module): return (m[0][3], m[1])

In [36]:
# #export
# def _densedunet(arch, data, config):
#     "Returns a resdunet model for a arch from data and final config"
#     pretrained, cut = config.pop("pretrained"), config.pop("cut")
#     body = create_body(arch, pretrained, cut)[0]
#     densenet_children = list(body.children())
#     new_body = nn.Sequential(nn.Sequential(*densenet_children[:4]),
#                                nn.Sequential(*densenet_children[4:6]),
#                                nn.Sequential(*densenet_children[6:8]),
#                                nn.Sequential(*densenet_children[8:10]),
#                                nn.Sequential(*densenet_children[10:]))
#     try:    size = data.train_ds[0][0].size
#     except: size = next(iter(data.train_dl))[0].shape[-2:]
#     model = DynamicUnet(new_body, n_classes=data.c, img_size=size, **config)
#     return model

In [37]:
# #export
# def densedunet121(data, config): return _densedunet(densenet121, data, config)
# model_funcs['densedunet121'] = densedunet121
# default_configs['densedunet121'] = dunet_config
# splits['densedunet121'] = _densedunet_split

# def densedunet161(data, config): return _densedunet(densenet161, data, config)
# model_funcs['densedunet161'] = densedunet161
# default_configs['densedunet161'] = dunet_config
# splits['densedunet161'] = _densedunet_split

# def densedunet169(data, config): return _densedunet(densenet169, data, config)
# model_funcs['densedunet169'] = densedunet169
# default_configs['densedunet169'] = dunet_config
# splits['densedunet169'] = _densedunet_split

# def densedunet201(data, config): return _densedunet(densenet201, data, config)
# model_funcs['densedunet201'] = densedunet201
# default_configs['densedunet201'] = dunet_config
# splits['densedunet201'] = _densedunet_split


### `SE-ResDUneXt [50,101]`

In [26]:
#export
from fastai.vision.models.cadene_models import se_resnext50_32x4d, se_resnext101_32x4d

In [27]:
#export
_se_meta = model_meta[se_resnext50_32x4d]
_se_cut, _se_split = _se_meta['cut'], _se_meta['split']

In [28]:
#export
def _seresdunext(arch, data, config):
    "Returns a resdunet model for a arch from data and final config"
    pretrained = config.pop("pretrained")
    try:    size = data.train_ds[0][0].size
    except: size = next(iter(data.train_dl))[0].shape[-2:]
    body = create_body(arch, pretrained, cut=_se_cut)
    model = DynamicUnet(body, n_classes=data.c, img_size=size, **config)
    return model

In [29]:
#export
def seresdunext50(data, config): return _seresdunext(se_resnext50_32x4d, data, config)
model_funcs['seresdunext50'] = seresdunext50
default_configs['seresdunext50'] = dunet_config
splits['seresdunext50'] = _se_split

def seresdunext101(data, config): return _seresdunext(se_resnext101_32x4d, data, config)
model_funcs['seresdunext101'] = seresdunext101
default_configs['seresdunext101'] = dunet_config
splits['seresdunext101'] = _se_split

### `DeepLab v3+ [ResNet50, ResNet101, seresnext50, seresnext101]`

https://github.com/yelanlan/Pneumothorax-Segmentation-2nd-place-solution/blob/bf99230deffdd813fe730ddeaed6822ba37193df/semantic_segmentation/network/deepv3.py

In [30]:
# from torchvision.models.segmentation import deeplabv3_resnet50, deeplabv3_resnet101

In [31]:
#export
_body_config = {"pretrained":True} 
_deeplab_config = {'variant':'D', 'skip':'m1', 'skip_num':48}
deeplab_config = {**_body_config, **_deeplab_config}

In [32]:
#export
class _AtrousSpatialPyramidPoolingModule(nn.Module):
    """
    operations performed:
      1x1 x depth
      3x3 x depth dilation 6
      3x3 x depth dilation 12
      3x3 x depth dilation 18
      image pooling
      concatenate all together
      Final 1x1 conv
    """

    def __init__(self, in_dim, reduction_dim=256, output_stride=16, rates=(6, 12, 18)):
        super(_AtrousSpatialPyramidPoolingModule, self).__init__()

        # Check if we are using distributed BN and use the nn from encoding.nn
        # library rather than using standard pytorch.nn

        if output_stride == 8:
            rates = [2 * r for r in rates]
        elif output_stride == 16:
            pass
        else:
            raise 'output stride of {} not supported'.format(output_stride)

        self.features = []
        # 1x1
        self.features.append(
            nn.Sequential(nn.Conv2d(in_dim, reduction_dim, kernel_size=1, bias=False),
                          nn.BatchNorm2d(reduction_dim), nn.ReLU(inplace=True)))
        # other rates
        for r in rates:
            self.features.append(nn.Sequential(
                nn.Conv2d(in_dim, reduction_dim, kernel_size=3,
                          dilation=r, padding=r, bias=False),
                nn.BatchNorm2d(reduction_dim),
                nn.ReLU(inplace=True)
            ))
        self.features = torch.nn.ModuleList(self.features)

        # img level features
        self.img_pooling = nn.AdaptiveAvgPool2d(1)
        self.img_conv = nn.Sequential(
            nn.Conv2d(in_dim, reduction_dim, kernel_size=1, bias=False),
            nn.BatchNorm2d(reduction_dim), nn.ReLU(inplace=True))
        
    def forward(self, x):
        x_size = x.size()

        img_features = self.img_pooling(x)
        img_features = self.img_conv(img_features)
        img_features = _Upsample(img_features, x_size[2:])
        out = img_features

        for f in self.features:
            y = f(x)
            out = torch.cat((out, y), 1)
        return out
    
def _Upsample(x, size):
    """
    Wrapper Around the Upsample Call
    """
    return nn.functional.interpolate(x, size=size, mode='bilinear', align_corners=True)

In [33]:
#export
class _DeepV3Plus(nn.Module):
    """
    Implement DeepLab-V3 model
    A: stride8
    B: stride16
    with skip connections
    """
    def __init__(self, num_classes, backbone='seresnext-50', pretrained=True, variant='D', 
                 skip='m1', skip_num=48):
        super(_DeepV3Plus, self).__init__()
        
        self.variant, self.skip, self.skip_num = variant, skip, skip_num
        
        if backbone == 'seresnext50':
            body = create_body(se_resnext50_32x4d, pretrained)
        elif backbone == 'seresnext101':
            body = create_body(se_resnext101_32x4d, pretrained)
        elif backbone == 'resnet50':
            body = create_body(resnet50, pretrained)
            body = nn.Sequential(nn.Sequential(body[:4]), *body[4:])
        elif backbone == 'resnet101':
            body = create_body(resnet101, pretrained)
            body = nn.Sequential(nn.Sequential(body[:4]), *body[4:])
        else:
            raise ValueError("Not a valid network arch")
            
        self.body = body

        if self.variant == 'D':
            for n, m in self.body[3].named_modules():
                if 'conv2' in n:
                    m.dilation, m.padding, m.stride = (2, 2), (2, 2), (1, 1)
                elif 'downsample.0' in n:
                    m.stride = (1, 1)
            for n, m in self.body[4].named_modules():
                if 'conv2' in n:
                    m.dilation, m.padding, m.stride = (4, 4), (4, 4), (1, 1)
                elif 'downsample.0' in n:
                    m.stride = (1, 1)
        elif self.variant == 'D16':
            for n, m in self.body[4].named_modules():
                if 'conv2' in n:
                    m.dilation, m.padding, m.stride = (2, 2), (2, 2), (1, 1)
                elif 'downsample.0' in n:
                    m.stride = (1, 1)
        else:
            # raise 'unknown deepv3 variant: {}'.format(self.variant)
            print("Not using Dilation ")

        self.aspp = _AtrousSpatialPyramidPoolingModule(2048, 256, output_stride=8)

        if self.skip == 'm1':
            self.bot_fine = nn.Conv2d(256, self.skip_num, kernel_size=1, bias=False)
        elif self.skip == 'm2':
            self.bot_fine = nn.Conv2d(512, self.skip_num, kernel_size=1, bias=False)
        else:
            raise Exception('Not a valid skip')

        self.bot_aspp = nn.Conv2d(1280, 256, kernel_size=1, bias=False)

        self.final = nn.Sequential(
            nn.Conv2d(256 + self.skip_num, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, num_classes, kernel_size=1, bias=False))
        
        # init weights and biases
        for m in [self.aspp, self.bot_aspp, self.bot_fine, self.final]:
            apply_init(m, nn.init.kaiming_normal_)
        
        
    def forward(self, x, gts=None):

        x_size = x.size()  # 800
        x0 = self.body[0](x)  # 400
        x1 = self.body[1](x0)  # 400
        x2 = self.body[2](x1)  # 100
        x3 = self.body[3](x2)  # 100
        x4 = self.body[4](x3)  # 100
        xp = self.aspp(x4)

        dec0_up = self.bot_aspp(xp)
        if self.skip == 'm1':
            dec0_fine = self.bot_fine(x1)
            dec0_up = _Upsample(dec0_up, x1.size()[2:])
        else:
            dec0_fine = self.bot_fine(x2)
            dec0_up = _Upsample(dec0_up, x2.size()[2:])

        dec0 = [dec0_fine, dec0_up]
        dec0 = torch.cat(dec0, 1)
        dec1 = self.final(dec0)
        main_out = _Upsample(dec1, x_size[2:])
        
        return main_out

In [34]:
#export
def _deeplabv3(arch_name, data, config):
    "Returns a resdunet model for a arch from data and final config"
    pretrained = config.pop("pretrained")
    model = _DeepV3Plus(data.c, arch_name, pretrained=pretrained, **config)
    return model

In [35]:
#export
def _deeplab_split(m:nn.Module): return (m.body[3], m.aspp)

In [36]:
#export
def deeplabv3res50(data, config): return _deeplabv3("resnet50", data, config)
model_funcs['deeplabv3res50'] = deeplabv3res50
default_configs['deeplabv3res50'] = deeplab_config
splits['deeplabv3res50'] = _deeplab_split

def deeplabv3res101(data, config): return _deeplabv3("resnet101", data, config)
model_funcs['deeplabv3res101'] = deeplabv3res101
default_configs['deeplabv3res101'] = deeplab_config
splits['deeplabv3res101'] = _deeplab_split

def deeplabv3seresnext50(data, config): return _deeplabv3("seresnext50", data, config)
model_funcs['deeplabv3seresnext50'] = deeplabv3seresnext50
default_configs['deeplabv3seresnext50'] = deeplab_config
splits['deeplabv3seresnext50'] = _deeplab_split

def deeplabv3seresnext101(data, config): return _deeplabv3("seresnext101", data, config)
model_funcs['deeplabv3seresnext101'] = deeplabv3seresnext101
default_configs['deeplabv3seresnext101'] = deeplab_config
splits['deeplabv3seresnext101'] = _deeplab_split

### `tests`

In [38]:
from local.test import *
model_funcs, splits

({'smpunet': <function __main__.smpunet(data, config)>,
  'smpfpn': <function __main__.smpfpn(data, config)>,
  'smplinknet': <function __main__.smplinknet(data, config)>,
  'smppspnet': <function __main__.smppspnet(data, config)>,
  'resdunet18': <function __main__.resdunet18(data, config)>,
  'resdunet34': <function __main__.resdunet34(data, config)>,
  'resdunet50': <function __main__.resdunet50(data, config)>,
  'resdunet101': <function __main__.resdunet101(data, config)>,
  'resdunet152': <function __main__.resdunet152(data, config)>,
  'seresdunext50': <function __main__.seresdunext50(data, config)>,
  'seresdunext101': <function __main__.seresdunext101(data, config)>,
  'deeplabv3res50': <function __main__.deeplabv3res50(data, config)>,
  'deeplabv3res101': <function __main__.deeplabv3res101(data, config)>,
  'deeplabv3seresnext50': <function __main__.deeplabv3seresnext50(data, config)>,
  'deeplabv3seresnext101': <function __main__.deeplabv3seresnext101(data, config)>},
 {'smpu

#### `test: smpunet`

In [39]:
model, split_fn, config = get_model("smpunet", data, {})

In [40]:
_test_model(model, data, split_fn)

epoch,train_loss,valid_loss,foreground_acc,time
0,1.027871,0.661890,0.843014,00:40
1,0.677247,0.549173,0.868110,00:35


epoch,train_loss,valid_loss,foreground_acc,time
0,0.617935,0.513590,0.878561,00:35
1,0.592622,0.477353,0.884004,00:35


epoch,train_loss,valid_loss,foreground_acc,time
0,0.546016,0.499873,0.873320,00:44
1,0.529006,0.446727,0.887710,00:44


0.8877096772193909


#### `test: smpfpn`

In [41]:
model, split_fn, config = get_model("smpfpn", data, {})

In [42]:
_test_model(model, data, split_fn)

epoch,train_loss,valid_loss,foreground_acc,time
0,1.010108,0.680289,0.829991,00:17
1,0.654420,0.507232,0.857251,00:16


epoch,train_loss,valid_loss,foreground_acc,time
0,0.585172,0.530360,0.855793,00:15
1,0.572407,0.485526,0.859105,00:15


epoch,train_loss,valid_loss,foreground_acc,time
0,0.513032,0.413255,0.873264,00:18
1,0.497358,0.417243,0.868900,00:17


0.8689004778862


#### `test: smplinknet`

In [44]:
model, split_fn, config = get_model("smplinknet", data, {})

In [45]:
_test_model(model, data, split_fn)

epoch,train_loss,valid_loss,foreground_acc,time
0,1.300285,0.808546,0.836848,00:17
1,0.836103,0.701907,0.847225,00:16


epoch,train_loss,valid_loss,foreground_acc,time
0,0.763721,0.655002,0.851313,00:16
1,0.709223,0.614126,0.852938,00:17


epoch,train_loss,valid_loss,foreground_acc,time
0,0.684031,0.583277,0.855636,00:19
1,0.656221,0.543988,0.858152,00:19


0.8581520915031433


#### `test: smppspnet`

In [46]:
model, split_fn, config = get_model("smppspnet", data, {})

In [47]:
_test_model(model, data, split_fn)

epoch,train_loss,valid_loss,foreground_acc,time
0,0.907720,0.563953,0.846346,00:13
1,0.622648,0.478826,0.863709,00:12


epoch,train_loss,valid_loss,foreground_acc,time
0,0.581823,0.456502,0.872792,00:12
1,0.548576,0.437448,0.872428,00:13


epoch,train_loss,valid_loss,foreground_acc,time
0,0.544534,0.438770,0.870150,00:13
1,0.518414,0.434794,0.871329,00:13


0.8713288307189941


#### `test: resdunet`

In [52]:
model, split_fn, config = get_model(name="resdunet18", data=data, config={"self_attention":False})

In [53]:
_test_model(model, data, split_fn)

epoch,train_loss,valid_loss,foreground_acc,time
0,1.221925,0.987110,0.745205,00:28
1,0.758828,0.637509,0.838978,00:25


epoch,train_loss,valid_loss,foreground_acc,time
0,0.636126,0.524207,0.853481,00:26
1,0.573152,0.490320,0.856292,00:26


epoch,train_loss,valid_loss,foreground_acc,time
0,0.510775,0.497969,0.860382,00:27
1,0.501385,0.397208,0.884266,00:27


0.8842664957046509


#### `test: seresdunext`

In [65]:
data = _get_test_data(224)

In [66]:
model, split_fn, config = get_model(name="seresdunext50", data=data, config={"self_attention":True})

In [67]:
_test_model(model, data, split_fn)

epoch,train_loss,valid_loss,foreground_acc,time
0,4.702204,1.137622,0.641244,02:05
1,0.956674,0.642755,0.826381,02:01


epoch,train_loss,valid_loss,foreground_acc,time
0,0.705070,0.585960,0.841091,02:04
1,0.622628,0.479730,0.858037,02:04


epoch,train_loss,valid_loss,foreground_acc,time
0,0.552522,0.442669,0.868224,02:05
1,0.503666,0.390541,0.881810,02:05


0.8818103075027466


#### `test: deeplabv3+`

In [68]:
model, split_fn, config = get_model(name="deeplabv3res50", data=data, config={})

In [69]:
_test_model(model, data, split_fn)

epoch,train_loss,valid_loss,foreground_acc,time
0,0.867689,0.661466,0.827699,00:26
1,0.632980,0.507957,0.852487,00:22


epoch,train_loss,valid_loss,foreground_acc,time
0,0.562511,0.482732,0.869619,00:26
1,0.562631,0.426355,0.876510,00:25


epoch,train_loss,valid_loss,foreground_acc,time
0,0.481868,0.401254,0.883670,00:26
1,0.466730,0.387020,0.885651,00:26


0.8856508731842041


### export

In [71]:
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_script.ipynb.
Converted 02_scheduler.ipynb.
Converted 03_callbacks.ipynb.
Converted 04_optimizers_optimizers.ipynb.
Converted 10_segmentation_dataset.ipynb.
Converted 11_segmentation_losses_mulitlabel.ipynb.
Converted 11b_segmentation_losses_binary.ipynb.
Converted 12_segmentation_metrics.ipynb.
Converted 13_segmentation_models.ipynb.
Converted 14_segmentation_postprocess.ipynb.
Converted 15_segmentation_tta.ipynb.
Converted 16_segmentation_utils.ipynb.
Converted 20_classification_dataset.ipynb.
Converted 21_classification_losses.ipynb.
Converted 23_classification_models.ipynb.
Converted classification_training.ipynb.
Converted segmentation_training.ipynb.


### fin